# Estimating the required number of telomeres

We can naturally ask the question "how many telomeres are required to measure the mean to a given level of uncertainty?"

We can make this estimation as follows. Let's start by assuming that the only source of error is undersampling the distribution of telomeres. We can also make the assumption that the distribution of telomere sizes is Gaussian, which is not strictly correct but also not entirely unreasonable (the distributions have a slight left-ward skew). Finally, we know from our TRF experiments that the uncertainty in the mean values is roughly +/- 5 nm.

Assuming a population standard deviation of $\sigma$, the sample standard deviation is 

$$ s = \frac{\sigma}{N} $$

with $N$ representing the number of telomeres recorded in an experiment. Since $\sigma$ is fixed, we can write

$$ s_{1} \sqrt{ N_{1} } = s_{2} \sqrt{ N_{2} } $$

Letting $N_{1} = 200$ and $s_{1} = 5 \; nm$ (which are typicaly values from the TRF knockdown experiments, we can solve the above equation for $s_{2} = 1 \; nm$ and find that

$$ N_{2} = 5000 \quad \left(s_{2} = 1 \; nm \right) $$

This means we would roughly need to measure 5000 telomeres to bring the uncertainty in the mean value down to $ \pm 1 \; nm$ if the only source of errors was undersampling. Since other errors will be present, **I think we should aim to measure 10,000 telomeres** in an experiment to bring down the uncertainty in the mean.

# Confidence intervals for the wild type measurements

Let's test this estimation by computing the standard errors and confidence intervals for the wild type experiments. In these experiments, between 1000 and 1500 telomeres were measured, so their uncertainties should be between 1 and 5 $nm$.

In [3]:
# Read the wild type distributions into the environment
%pylab

pathToDists = '../../saved_distrs/'

hl = np.loadtxt(pathToDists + 'Original_Data_L_dataset_RgTrans')
hs = np.loadtxt(pathToDists + 'Original_Data_S_dataset_RgTrans')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [11]:
print('Hela L, mean: {0:.1f}'.format(np.mean(hl)))
print('Hela L, std.: {0:.1f}'.format(np.std(hl)))
print('Hela L,    N: {0:.1f}'.format(hl.size))
print('--------------------')
print('Hela S, mean: {0:.1f}'.format(np.mean(hs)))
print('Hela S, std.: {0:.1f}'.format(np.std(hs)))
print('Hela S,    N: {0:.1f}'.format(hs.size))

Hela L, mean: 105.1
Hela L, std.: 29.0
Hela L,    N: 1754.0
--------------------
Hela S, mean: 80.2
Hela S, std.: 19.1
Hela S,    N: 1041.0


In [15]:
stdErrL = np.std(hl) / np.sqrt(hl.size)
print('Hela L, standard error of the mean: {0:.2f} nm'.format(stdErrL))

stdErrS = np.std(hs) / np.sqrt(hs.size)
print('Hela S, standard error of the mean: {0:.2f} nm'.format(stdErrS))

Hela L, standard error of the mean: 0.69
Hela S, standard error of the mean: 0.59


Surprisingly, these numbers are quite small relative to the $5 \; nm$ I quoted above. I derived this $\pm 5 \; nm$ value from the *confidence intervals for the difference of means* from the TRF1 experiments, so really I should ask what is the standard error for the difference of means.

In [16]:
stdErrDiffL = np.sqrt(2) * stdErrL
print('Hela L, estimate of standard error for difference of means: {0:.2f} nm'.format(stdErrDiffL))

stdErrDiffS = np.sqrt(2) * stdErrS
print('Hela S, estimate of standard error for difference of means: {0:.2f} nm'.format(stdErrDiffS))

Hela L, estimate of standard error for difference of means: 0.98 nm
Hela S, estimate of standard error for difference of means: 0.84 nm


These numbers are still relatively small. Let's see what the widths of the 95% confidence intervals would be by bootstrap resampling the difference between two resampled means from each distribution.

## Confidence Intervals

In [17]:
from sigTestHelpers import bootstrapDiff

In [20]:
# bootstrapDiff creates 100,000 resamples by default
(CIl, pValuel) = bootstrapDiff(hl, hl)
(CIs, pValues) = bootstrapDiff(hs, hs)

In [23]:
CIWidthL = CIl[1] - CIl[0]
print('Hela L, width of 95% confidence interval: {0:.2f}'.format(CIWidthL))

CIWidthS = CIs[1] - CIs[0]
print('Hela S, width of 95% confidence interval: {0:.2f}'.format(CIWidthS))

Hela L, width of 95% confidence interval: 3.83
Hela S, width of 95% confidence interval: 3.28


These numbers are much more in line with the estimates I made above and are probably more indicative of the . For the TRF knockdowns, the 95% confidence interval was roughly $\pm 5 \; nm$. Each sample had roughly 200 telomeres.

In the wild type experiments, which have between 5 and 7 times more telomeres per sample, the width of the confidence intervals have shrunk to $3.8 \; nm$ and $3.3 \; nm$, respectively.

**So if we want to measure a difference in mean telomere sizes with an uncertainty (as measured by the width of the 95% confidence interval) that is about $\pm 1 \; nm$, between 5000 and 10,000 telomeres will be required.**

We should probably aim for 10,000 due to other errors that can arise besides simple subsampling.

In [26]:
5 / np.sqrt(200)

0.35355339059327373

In [29]:
3.5 / np.sqrt(1000)

0.11067971810589328